# YamboWorkflow: a full DFT+MBPT flow including spin-orbit coupling

This represents the main workflow of the plugin. 
It allows the user to run automatically a complete DFT+GW set of simulations. In order to do this, both the `YamboRestart` and the `PwBaseWorkChain` are called. In this way, 
automatic error handling for both DFT and MBPT part is enabled.

We are going to pre-populate the input parameters in the same way as done in the previous tutorial, withing the `YamboRestart` workchain. Indeed, in every workchain of the plugin, 
a corresponding `get_builder_from_protocol` method is implemented.

NB: it is also possible to run BSE@GW (or just DFT), as you will see in a following dedicated tutorial.

In [ ]:
from aiida import orm, load_profile
load_profile()

from aiida.plugins import WorkflowFactory
from aiida.orm import QueryBuilder
from aiida.engine import submit

from aiida_quantumespresso.common.types import ElectronicType

import yaml

qb = QueryBuilder()
qb.append(orm.Group, filters={'label': 'Silicon/bulk'}, tag='group')
qb.append(orm.StructureData, with_group='group')

loaded_structure_id = qb.all()[0][0].pk

# Read YAML file
with open("../configuration/codes_localhost.yaml", 'r') as stream:
    codes = yaml.safe_load(stream)
    
with open("../configuration/resources_localhost.yaml", 'r') as stream:
    resources = yaml.safe_load(stream)
    
options = {
    'pseudo_family':"PseudoDojo/0.4/PBE/SR/standard/upf",
    'protocol':'fast',
    #'parent_id':274, #not necessary to set; if you want it, take ytheour previously nscf id (pk) to skip the DFT part.
    'structure_id':loaded_structure_id,
}

### Overrides + Spin Orbit Coupling

As in the previous examples (see e.g. then  `YamboRestart` notebook), it is possible to modify the default inputs also during the builder creation phase, so not a posteriori. This can be done by using overrides:

In [12]:
overrides_scf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/FR/standard/upf", 
        'pw':{
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{
                    "noncolin": True,
                    "lspinorb": True,
                },
                'ELECTRONS':{},
            },
            
        'metadata':resources,
        },
    }

overrides_nscf = {
        'pseudo_family': "PseudoDojo/0.4/PBE/FR/standard/upf", 
        'pw': {
            'parameters':{
                'CONTROL':{}, #not needed if you don't override something
                'SYSTEM':{
                    "noncolin": True,
                    "lspinorb": True,
                },
                'ELECTRONS':{'diagonalization':'cg'},
            },
             'metadata':resources,
    },
}

overrides_yambo = {
        "yambo": {
            "parameters": {
                "arguments": [
                    "rim_cut",
                ],
                "variables": {
                    "NGsBlkXp": [4, "Ry"],
                    "FFTGvecs": [20, "Ry"],
                },
            },
        'metadata':{
                    'options':{
                    'max_wallclock_seconds': 60*60, # in seconds
                    'resources': {
                            "num_machines": 1, # nodes
                            "num_mpiprocs_per_machine": 16, # MPI per nodes
                            "num_cores_per_mpiproc": 1, # OPENMP
                        },
                    'prepend_text': u"export OMP_NUM_THREADS="+str(1), # if needed, i.e. in PBS/Torque 
                    #'account':'project_name',
                    'queue_name':'s3par',
                    #'qos':'',
                                    },
                    },
        },
    
}

overrides = {
    'yres': overrides_yambo,
    'nscf': overrides_nscf,
    'scf': overrides_scf
    
}


In [13]:
YamboWorkflow = WorkflowFactory('yambo.yambo.yambowf')

builder = YamboWorkflow.get_builder_from_protocol(
            pw_code = options['pwcode_id'],
            preprocessing_code = options['yamboprecode_id'],
            code = options['yambocode_id'],
            protocol=options['protocol'],
            protocol_qe=options['protocol'],
            structure= orm.load_node(options['structure_id']),
            overrides=overrides,
            pseudo_family= options['pseudo_family'],
            #parent_folder=orm.load_node(options['parent_id']).outputs.remote_folder,
            electronic_type=ElectronicType.INSULATOR, #default is METAL: in that case, smearing is used
            calc_type='gw', #or 'bse'; default is 'gw'
)

builder.scf.pw.metadata.options = resources

builder.nscf.pw.metadata.options = builder.scf.pw.metadata.options
builder.yres.yambo.metadata.options = builder.scf.pw.metadata.options

Summary of the main inputs:
BndsRnXp = 200
GbndRnge = 200
NGsBlkXp = 4 Ry
FFTGvecs = 20 Ry


kpoint mesh for nscf: [6, 6, 2]


In [14]:
builder.nscf.pw.parameters.get_dict()

{'CONTROL': {'calculation': 'nscf',
  'forc_conv_thr': 0.001,
  'tprnfor': True,
  'tstress': True,
  'etot_conv_thr': 0.0004},
 'SYSTEM': {'nosym': False,
  'occupations': 'fixed',
  'noncolin': True,
  'lspinorb': True,
  'ecutwfc': 84.0,
  'ecutrho': 336.0,
  'force_symmorphic': True,
  'nbnd': 200},
 'ELECTRONS': {'electron_maxstep': 80,
  'mixing_beta': 0.4,
  'diagonalization': 'cg',
  'conv_thr': 1.6e-09}}

In [15]:
builder.yres.yambo.metadata.options

{'stash': {}, 'resources': {'num_machines': 1, 'num_mpiprocs_per_machine': 16, 'num_cores_per_mpiproc': 1}, 'max_wallclock_seconds': 3600, 'withmpi': True, 'prepend_text': 'export OMP_NUM_THREADS=1', 'queue_name': 's3par'}

In [16]:
builder.nscf.pw.parameters.get_dict()['ELECTRONS']['diagonalization']

'cg'

In [17]:
family = orm.load_group("PseudoDojo/0.4/PBE/FR/standard/upf")
#builder.<sublevels_up_to .pw>.pseudos = family.get_pseudos(structure=structure) 
builder.scf.pw.pseudos = family.get_pseudos(structure=orm.load_node(loaded_structure_id)) 
builder.nscf.pw.pseudos = family.get_pseudos(structure=orm.load_node(loaded_structure_id)) 

In [18]:
builder.additional_parsing = orm.List(list=['gap_GG','gap_'])

In [20]:
run = None

In [21]:
if run:
    print('run is already running -> {}'.format(run.pk))
    print('sure that you want to run again?, if so, copy the else instruction in the cell below and run!')
else:
    run = submit(builder)

print(run)

/scratch/mbonacci/mambaforge/envs/minitutorial/lib/python3.10/site-packages/aiida_quantumespresso/calculations/pw.py:211: UserWarning: `parent_folder` not provided for `nscf` calculation. For work chains wrapping this calculation, you can disable this warning by excluding the `parent_folder` when exposing the inputs of the `PwCalculation`.
  warnings.warn(


uuid: 643296f7-6077-4717-ba4f-a29fb8cd7b25 (pk: 5750) (aiida.workflows:yambo.yambo.yambowf)


# Inspecting the outputs

suppose that your calculation completed successfully, then you can access the outputs via the output method of the run instance. All the outputs of YamboRestart and YamboCalculation are inherited

In [24]:
run.is_finished_ok

True

In [25]:
run.outputs.output_ywfl_parameters.get_dict()

{'gap_': 5.3789906471538,
 'homo': -0.20861223974703,
 'lumo': 5.1703784074068,
 'gap_GG': 7.3148622832797,
 'homo_G': -1.5859060006194,
 'lumo_G': 5.7289562826604,
 'gap_dft': 4.2862447410166,
 'homo_dft': -8.5151106119156e-06,
 'lumo_dft': 4.286236225906,
 'gap_GG_dft': 6.7760488920897,
 'homo_G_dft': -1.2751118633211,
 'lumo_G_dft': 5.5009370287687}

Moreover, the information extracted from the nscf step are stored in the `nscf_mapping` output node:

In [26]:
run.outputs.nscf_mapping.get_dict()

{'soc': True,
 'gap_': [[14, 14, 15, 15], [7, 7, 17, 17]],
 'gap_GG': [[1, 1, 15, 15], [1, 1, 17, 17]],
 'homo_k': 14,
 'lumo_k': 7,
 'valence': 15,
 'gap_type': 'indirect',
 'conduction': 17,
 'nscf_gap_eV': 4.286,
 'dft_predicted': 'semiconductor/insulator',
 'number_of_kpoints': 14,
 'magnetic_calculation': False}

Where you can see that indeed we performed a calculation with spin-orbit coupling (soc=True).